In [1]:
# Instalar o SDK do Google
!pip install -q -U google-generativeai

# Importar as bibliotecas necessárias
import google.generativeai as genai
import os # Usaremos para variáveis de ambiente se necessário
from google.colab import userdata # Para buscar a API Key de forma segura no Colab

In [2]:
# Configure sua API Key
# 1. Vá em "Get API Key" no Google AI Studio (ou na lateral esquerda do Colab no ícone de chave "Secrets")
# 2. Crie ou use uma API Key existente.
# 3. No Colab, clique no ícone de chave 🔑 (Segredos) na barra lateral esquerda.
# 4. Clique em "+ Adicionar novo secret".
# 5. No campo "Nome", digite exatamente: GOOGLE_API_KEY
# 6. No campo "Valor", cole a sua chave de API.
# 7. Ative o botão "Acesso do notebook".
# 8. Agora, podemos buscar essa chave de forma segura:

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: Secret 'GOOGLE_API_KEY' não encontrado.")
    print("Por favor, configure a API Key nos segredos do Colab como instruído acima.")
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API Key: {e}")

API Key configurada com sucesso!


In [3]:
# Configurações do modelo Gemini que usaremos (podemos ajustar depois)
generation_config = {
    "candidate_count": 1, # Gera apenas uma resposta
    "temperature": 0.7,   # Um valor entre 0 e 1. Mais alto = mais criativo/aleatório.
                          # Para um detetive, talvez um valor mais baixo seja melhor se usarmos IA para análise.
                          # Por enquanto, para regras, isso não afeta.
}

safety_settings = {
    # Configurações de segurança para evitar conteúdo prejudicial
    # Ajuste conforme necessário, mas os padrões costumam ser bons.
}

# Inicializar o modelo Generativo
# Usaremos o gemini-1.0-pro por enquanto, que é um bom modelo geral.
# Se precisarmos de algo mais rápido e simples, poderíamos usar o "gemini-1.0-pro-001" ou o flash.
try:
    model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings)
    print("Modelo Gemini inicializado com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao inicializar o modelo: {e}")

Modelo Gemini inicializado com sucesso!


In [4]:
def chatbot_detetive_de_golpes():
    print("🤖 Olá! Sou o Detetive de Golpes IA.")
    print("🕵️ Minha missão é te ajudar a identificar possíveis golpes em mensagens!")
    print("Por favor, cole abaixo a mensagem suspeita que você recebeu ou descreva a situação.")
    print("Lembre-se: Eu sou um assistente baseado em regras e meu conhecimento é limitado.")
    print("Sempre desconfie e, na dúvida, não clique e não forneça dados!")
    print("------------------------------------------------------------------------------------")

    mensagem_usuario = input("Digite ou cole a mensagem suspeita aqui: ")
    mensagem_lower = mensagem_usuario.lower() # Converter para minúsculas para facilitar a busca

    # --- Listas de Detecção ---
    # Você pode (e deve!) adicionar mais palavras e padrões aqui com o tempo!
    # Quanto mais completo, melhor o detetive.

    palavras_chave_suspeitas = [
        "parabéns você ganhou", "você foi sorteado", "prêmio incrível", "oferta exclusiva e limitada",
        "clique aqui urgente", "regularize sua situação imediatamente", "sua conta foi bloqueada",
        "última chance", "não perca", "dinheiro fácil", "renda extra garantida", "investimento seguro e rápido",
        "desconto imperdível só hoje", "problemas com sua entrega", "atualize seus dados agora",
        "suporte técnico informa", "banco central comunica", "receita federal intimação",
        "seu nome foi protestado", "ameaça de negativação", "valor bloqueado", "liberação de acesso"
    ]

    frases_comuns_golpe = [
        "para liberar seu prêmio, pague uma pequena taxa",
        "confirme seus dados bancários para continuar",
        "envie seus documentos para este email",
        "seu parente precisa de ajuda financeira urgente", # Golpe do falso parente
        "falso empréstimo consignado", "oportunidade de emprego falsa"
    ]

    # Padrões de links que podem ser suspeitos
    # Adicionar mais TLDs (domínios de topo) estranhos ou encurtadores
    links_suspeitos_padroes = [
        "bit.ly/", "tinyurl.com/", ".xyz", ".club", ".online", ".top", ".live", ".info", ".site",
        "http://" # Links não seguros (HTTPS é o padrão hoje)
        # Poderíamos adicionar RegEx aqui para links com muitos números ou sequências aleatórias
    ]

    solicitacao_dados_sensiveis = [
        "cpf", "rg", "número do cartão", "validade do cartão", "código de segurança", "cvv",
        "senha do banco", "senha do aplicativo", "código de verificação", "token", "dados de acesso"
    ]

    erros_gramaticais_grosseiros_indicativos = [
        # Adicionar aqui exemplos de erros comuns que golpistas cometem
        # Ex: "voce", "problemaS", "acesso negado IMEDIATAMENTE" (ênfase exagerada)
        # Esta parte é mais subjetiva e pode precisar de IA para uma análise melhor,
        # mas podemos listar alguns padrões se você identificar.
    ]

    # --- Lógica de Análise ---
    alertas_encontrados = [] # Lista para guardar os motivos do alerta

    # 1. Verificar palavras-chave suspeitas
    for palavra in palavras_chave_suspeitas:
        if palavra in mensagem_lower:
            alertas_encontrados.append(f"Presença da expressão suspeita: '{palavra}'")

    # 2. Verificar frases comuns de golpe
    for frase in frases_comuns_golpe:
        if frase in mensagem_lower:
            alertas_encontrados.append(f"Contém uma frase comum em golpes: '{frase}'")

    # 3. Verificar padrões de links suspeitos
    for padrao_link in links_suspeitos_padroes:
        if padrao_link in mensagem_lower:
            if padrao_link == "http://" and "https://" not in mensagem_lower: # Alerta para http se não for https
                 alertas_encontrados.append(f"Uso de link não seguro (HTTP): um link seguro geralmente começa com HTTPS.")
            elif padrao_link != "http://":
                alertas_encontrados.append(f"Contém um padrão de link que pode ser suspeito: '{padrao_link}'. Verifique o remetente e o destino do link com MUITO cuidado antes de clicar.")

    # 4. Verificar solicitação direta de dados sensíveis
    for dado in solicitacao_dados_sensiveis:
        if dado in mensagem_lower:
            alertas_encontrados.append(f"Solicitação (direta ou indireta) de dados sensíveis: '{dado}'. Nunca forneça esses dados por mensagem!")

    # 5. (Opcional e Simples) Verificar se a mensagem é muito curta e só tem um link
    if len(mensagem_usuario.split()) < 5 and ("http://" in mensagem_lower or "https://" in mensagem_lower):
        alertas_encontrados.append("Mensagem muito curta contendo apenas um link, o que pode ser suspeito.")

    # --- Exibir Resultado da Análise ---
    print("------------------------------------------------------------------------------------")
    print("🔍 Resultado da Análise do Detetive de Golpes IA:")
    print("------------------------------------------------------------------------------------")

    if alertas_encontrados:
        print("🚨 ALERTA DE POTENCIAL GOLPE! 🚨")
        print("Identifiquei os seguintes pontos de atenção nesta mensagem:")
        # Usar set para mostrar cada tipo de alerta apenas uma vez, mesmo que acionado por múltiplas palavras
        for alerta_unico in sorted(list(set(alertas_encontrados))): # sorted para ordem alfabética
            print(f"- {alerta_unico}")
        print("\nRecomendações IMPORTANTES:")
        print("  - NÃO CLIQUE em links suspeitos.")
        print("  - NÃO FORNEÇA dados pessoais, senhas ou códigos.")
        print("  - NÃO FAÇA transferências ou pagamentos sob pressão.")
        print("  - DESCONFIE de promessas muito boas para ser verdade ou ameaças urgentes.")
        print("  - Se a mensagem parecer ser de uma empresa conhecida, entre em contato com a empresa por um canal OFICIAL (site, app oficial, telefone que VOCÊ procurou) para verificar a veracidade.")
    else:
        print("✅ Análise inicial concluída.")
        print("Com base nas minhas regras atuais, não identifiquei sinais ÓBVIOS de golpe nesta mensagem.")
        print("No entanto, isso NÃO garante que a mensagem seja 100% segura.")
        print("\Continue sempre alerta:")
        print("  - Verifique o remetente: Você conhece? Esperava essa mensagem?")
        print("  - Analise o contexto: Faz sentido essa solicitação ou informação?")
        print("  - Na dúvida, a melhor ação é NÃO INTERAGIR e buscar confirmação por outros meios.")

    print("------------------------------------------------------------------------------------")
    print("Lembre-se: a cautela é sua maior aliada contra golpes online!")
    print("------------------------------------------------------------------------------------")

In [ ]:
# Agora vamos chamar a função para iniciar o chatbot!
chatbot_detetive_de_golpes()

🤖 Olá! Sou o Detetive de Golpes IA.
🕵️ Minha missão é te ajudar a identificar possíveis golpes em mensagens!
Por favor, cole abaixo a mensagem suspeita que você recebeu ou descreva a situação.
Lembre-se: Eu sou um assistente baseado em regras e meu conhecimento é limitado.
Sempre desconfie e, na dúvida, não clique e não forneça dados!
------------------------------------------------------------------------------------
